## DHTML (Dynamic HTML)
차이점: POST -> AJAX / return HTML -> JSON

### 화면 깜빡임을 기준으로 체크함

In [1]:
# http://example.webscraping.com/places/default/search

In [6]:
from bs4 import BeautifulSoup
import re
from html import unescape
import requests
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}


def download(method, url, headers = None, param = None, data = None, timeout = 1, maxretries = 3):

    try:        
        resp = requests.request(method, url, params = param, data = data, headers = headers)
        resp.raise_for_status()
    
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, headers, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)

    return resp

In [8]:
url = "http://example.webscraping.com/places/default/search"
# param = {}
html = download("get", url)
dom = BeautifulSoup(html.text, "lxml")

In [16]:
dom.select(".results > a")

[]

### search_json 으로 한다

In [38]:
url = "http://example.webscraping.com/places/ajax/search.json"
param = {"search_term":"korea",
        "page_size":10,"page":0
        }
# http://example.webscraping.com/places/ajax/search.json?&search_term=korea&page_size=10&page=0
html = download("get", url, param)

InvalidHeader: Value for header {page_size: 10} must be of type str or bytes, not <class 'int'>

In [37]:
for _ in html.json()["records"]:
    print(_["country"])

### 안나온다...
### => 'Selenium'
브라우저의 행동을 자동화해줌 (웹 서비스/시스템의 단위테스트, 특정 시나리오의 테스트를 자동화하기 위해 만들어짐.)

In [39]:
from selenium import webdriver

In [40]:
# ChromeDriver

In [43]:
driver = webdriver.Chrome()

In [46]:
%%writefile account.json
{
    "id":"dvd8145","pw":"djrud12!"
}

Writing account.json


In [47]:
import json
with open("account.json") as fp:
    account = json.load(fp)

In [51]:
account["id"]

'dvd8145'

In [52]:
driver.get("https://www.naver.com")

In [53]:
driver.page_source           # dom

'<html lang="ko"><head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER">\n<meta name="robots" content="index,nofollow">\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="twitter:image" content="https://s.pstatic.net/static/www

In [64]:
driver.get("http://example.webscraping.com/places/default/search")

In [65]:
inputTag = driver.find_element_by_id("search_term")
inputTag

<selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="6f781c68-cf41-4e7e-afb1-243ba42ccb15")>

In [66]:
inputTag.send_keys("korea")

In [67]:
driver.find_element_by_xpath("//input[@type='submit']").click()

In [68]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#results")    #code

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

In [69]:
driver.find_element_by_css_selector("#results").text    #text

'North Korea\nSouth Korea'

In [81]:
# 네이버 로그인
driver.get("https://www.naver.com")

In [90]:
inputid = driver.find_element_by_id("id")
inputid.send_keys(account["id"])

inputpw = driver.find_element_by_id("pw")
inputpw.send_keys(account["pw"])

driver.find_element_by_xpath("//input[@class='btn_global']").click()

In [104]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="71c5f16fa44b8644bf414fa839b9ea9a")>

In [112]:
# 메일 제목 불러오기
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print("[css]", _.text)
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[XPATH]",_.text)

[css] [네이버페이]주문하신 상품이 발송되었습니다
[css] 새로운 환경에서 로그인 되었습니다.
[css] 던전앤파이터 아이디 휴면 전환 안내
[css] (광고) 나에게맞는 가발/모발이식 은 무엇? 신청만해도 300만원지원
[css] 어경권님께, SPC삼립 / 한국소비자원 등 오늘의 신입공채 속보!
[css] [네이버페이]결제하신 내역을 안내해드립니다
[css] [mgram] 지난번 진단으로부터 9개월이 경과했습니다(지난번: 2018년10월16일)
[css] 어경권님, ㈜원익머트리얼즈 등 오늘의 공채 소식입니다.
[css] [네이버페이]결제하신 내역을 안내해드립니다
[css] 새로운 환경에서 로그인 되었습니다.
[css] [넥슨] 넥슨 서비스 휴면 전환 안내
[css] (광고) 신용등급 상관없고, 저소득자도 가능한 정부지원자금 안내!
[css] 어경권님께, 넥슨네트웍스 / 인천교통공사 등 오늘의 신입공채 속보!
[css] 경력공채 일정이 도착했습니다 (평균연봉 포함)
[css] 어경권님, 아비코전자㈜ 등 오늘의 공채 소식입니다.
[css] (광고) 오늘(7/15)까지만★해커스어학원이 쏜다! 당장 모든 혜택 받으러 go ☞
[css] 새로운 환경에서 로그인 되었습니다.
[css] (광고)40세이상 42.3% 발생 백내장 검진은 받아보셨나요?
[css] 네이버 웹툰에서 결제한 내역을 안내해 드립니다.
[css] 네이버 웹툰에서 결제한 내역을 안내해 드립니다.
[css] 네이버 웹툰에서 결제한 내역을 안내해 드립니다.
[css] 네이버 웹툰에서 결제한 내역을 안내해 드립니다.
[css] 캘리포니아 백미 대왕
[css] 어경권님, 이연제약(주) 등 오늘의 공채 소식입니다.
[css] 이용 약관 업데이트
[css] [브리츠] 임시비밀번호 안내
[css] [브리츠] 임시비밀번호 안내
[css] 어경권님께, 한국전기안전공사 / 신용보증기금 등 오늘의 신입공채 속보!
[css] 어경권님, 한국호야전자㈜ 등 오늘의 공채 소식입니다.
[css] [학력별] 취준생이 선호하는 

In [114]:
driver.find_element_by_css_selector(".item_wrap.bu6").click()    #스팸메일함으로 가는 방법

In [115]:
# 메일 제목 불러오기
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print("[css]", _.text)
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[XPATH]",_.text)

[css] 여기서 돈 못따면 게임 접으세요 !하루에 1`000만 출금자 수두룩합니다(성인)
[css] (광고) 창업 때문에 고민하신다면 추천해 드립니다.
[css] [바른생각] 최대 38% 할인! 사은품까지 풍ㅡ성
[css] (광고) 강남역 18년 최대규모의 [임플란트] 치과 6월 이벤트 진행중
[css] 가입첫입금15％ 매일첫입금10％ 가입~쿠폰 2만 지급
[css] "2019 자율주행 기술 및 전동화 기술세미나"(07.23)
[css] (광고) 유망사업아이템 '8가지'을 소개드립니다.
[XPATH] 여기서 돈 못따면 게임 접으세요 !하루에 1`000만 출금자 수두룩합니다(성인)
[XPATH] (광고) 창업 때문에 고민하신다면 추천해 드립니다.
[XPATH] [바른생각] 최대 38% 할인! 사은품까지 풍ㅡ성
[XPATH] (광고) 강남역 18년 최대규모의 [임플란트] 치과 6월 이벤트 진행중
[XPATH] 가입첫입금15％ 매일첫입금10％ 가입~쿠폰 2만 지급
[XPATH] "2019 자율주행 기술 및 전동화 기술세미나"(07.23)
[XPATH] (광고) 유망사업아이템 '8가지'을 소개드립니다.


## 쿠키값 가져오기

In [118]:
for cookie in driver.get_cookies():
    print(cookie["name"],cookie["value"])

NIPD 1
NNB T2MLUURWQMXF2
NID_JKL u1zvKuDGvULE9PXpM0BkheCpAlkiBj9VijbIQS+lYCY=
nid_inf 1527198847
NMUSER uwKlKqEwaqbsKogrKxtZpHpvaAudFxnwFxMqKqKmaAblaqEwKogsKAnsadEsHo2ma9vsxonOaxRVad/s6xRVaqRVaqns1rejL9Us6xRVaqnD16lvpB2RKLl5WLl5MBp0bSloWrdnaAvmKARGW430DVdw7BF0Mre5pzJZDLl5WLl5MBp0bSloWrdnaAvmKAn=
NID_SES AAABlbWCvofIyhDO2xo3Mw1xyF0SldxxrMMNimeYoUoBfhohXLiXKAq1mEKJreFu0exH+15JKLQ3uWKv6FSjlhlPyKgbrbxnBNOUyR+CUAkzCRk/cLIw7hhssPtXByuxk1OHAkwSNKLjY1jrBcTRDefMCEiHEyRQz7IrHiryevJgip1PHZrODMrNcG4cQUrvfY2cQ3vzSDNL6HQAX9btcJX12vn+ClYwFlIRkJGvhFojYAzKSFurFhA9QBMRVVDvy9A435pxiyFWJAFsfZA5PC0cuEM0dvk6glXn4O/e7WIXQenHySgKwOrpJGqR/cGOEkjrVUy6/y17ypjnu3Nq4DPJDTKgjbHL7L/jHmNShE1WYDFYDOemaHt08zE/Wqih/Y2e3JaBAQvW8D8usGayAIYlONOX5SsTwMXupzA5bY/Uu5mUXUbXLN5lctFE6MBrUQhK5hf0RRLotITy4Zwbwr51H9LXQuqnWLEuKswGGc8upDr6XIC7m8AYSaene4lbppWTO4xs9/awb43YsRwzCTRiL+jcbQ2eyuAlNrNbVqs3waX7
NID_AUT qpFs/2NHCj2Ef4qIRaQX+31w4vlT+r0cKhrzUdnK0EiNGeZzJtBkkJMmYbZXsWB7


In [119]:
# 세션 만들기
import requests
session = requests.Session()

In [120]:
session.cookies

<RequestsCookieJar[]>

In [ ]:
session.cookies.set()    #session.cookies.set(name, value, **kwargs)

In [121]:
for cookie in driver.get_cookies():
    session.cookies.set(cookie["name"],cookie["value"])

In [122]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='NID_AUT', value='qpFs/2NHCj2Ef4qIRaQX+31w4vlT+r0cKhrzUdnK0EiNGeZzJtBkkJMmYbZXsWB7', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='u1zvKuDGvULE9PXpM0BkheCpAlkiBj9VijbIQS+lYCY=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_SES', value='AAABlbWCvofIyhDO2xo3Mw1xyF0SldxxrMMNimeYoUoBfhohXLiXKAq1mEKJreFu0exH+15JKLQ3uWKv6FSjlhlPyKgbrbxnBNOUyR+CUAkzCRk/cLIw7hhssPtXByuxk1OHAkwSNKLjY1jrBcTRDefMCEiHEyRQz7IrHiryevJgip1PHZrODMrNcG4cQUrvfY2cQ3vzSDNL6HQAX9btcJX12vn+ClYwFlIRkJGvhFojYAzKSFurFhA9QBMRVVDvy9A435px

In [124]:
html = session.get("https://mail.naver.com/#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%220%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D")
html.text

'\n\n\n<!DOCTYPE HTML>\n\n\t\n\t\n\t\n\t\n\t\n\t\t<html lang="ko">\n\t\n\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190611.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190611.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net" />\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190611.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\

In [126]:
htm = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=dvd8145")
for _ in htm.json()["mailData"]:
    print(_["subject"])

[네이버페이]주문하신 상품이 발송되었습니다
새로운 환경에서 로그인 되었습니다.
던전앤파이터 아이디 휴면 전환 안내
(광고) 나에게맞는 가발/모발이식 은 무엇? 신청만해도 300만원지원
어경권​님께, SPC삼립 / 한국소비자원 등 오늘의 신입공채 속보!
[네이버페이]결제하신 내역을 안내해드립니다
[mgram] 지난번 진단으로부터 9개월이 경과했습니다(지난번: 2018년10월16일)
어경권님, ㈜원익머트리얼즈 등 오늘의 공채 소식입니다.
[네이버페이]결제하신 내역을 안내해드립니다
새로운 환경에서 로그인 되었습니다.
[넥슨] 넥슨 서비스 휴면 전환 안내
(광고) 신용등급 상관없고, 저소득자도 가능한 정부지원자금 안내!
어경권​님께, 넥슨네트웍스 / 인천교통공사 등 오늘의 신입공채 속보!
경력공채 일정이 도착했습니다 (평균연봉 포함)
어경권님, 아비코전자㈜ 등 오늘의 공채 소식입니다.
(광고) 오늘(7/15)까지만★해커스어학원이 쏜다! 당장 모든 혜택 받으러 go ☞
새로운 환경에서 로그인 되었습니다.
(광고)40세이상 42.3% 발생 백내장 검진은 받아보셨나요?
네이버 웹툰에서 결제한 내역을 안내해 드립니다.
네이버 웹툰에서 결제한 내역을 안내해 드립니다.
네이버 웹툰에서 결제한 내역을 안내해 드립니다.
네이버 웹툰에서 결제한 내역을 안내해 드립니다.
캘리포니아 백미 대왕
어경권님, 이연제약(주) 등 오늘의 공채 소식입니다.
이용 약관 업데이트
[브리츠] 임시비밀번호 안내
[브리츠] 임시비밀번호 안내
어경권​님께, 한국전기안전공사 / 신용보증기금 등 오늘의 신입공채 속보!
어경권님, 한국호야전자㈜ 등 오늘의 공채 소식입니다.
[학력별] 취준생이 선호하는 튼튼한 중견기업!
[던전앤파이터] 마케팅 정보 수신동의 여부 확인 안내
(광고) 라식/라섹 필요하신 분들을 찾습니다~
경력공채 일정이 도착했습니다 (평균연봉 포함)
어경권​님께, 현대자동차 / 인천공항에너지 등 오늘의 신입공채 속보!
어경권님, ㈜이엠넷 등 오늘의 공채 소식입니다.
어경권님, (주)경보제약 등 오늘의

In [127]:
htm = requests.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=dvd8145")
htm.text    #쿠키값이 없어서(session이 아니니까) FAIL이 뜬다.

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [129]:
htms = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=dvd8145")
for _ in htms.json()["mailData"]:
    print(_['subject'])

여기서 돈 못따면 게임 접으세요 !하루에 1`000만 출금자 수두룩합니다(성인)
(광고) 창업 때문에 고민하신다면 추천해 드립니다.
[바른생각] 최대 38% 할인! 사은품까지 풍ㅡ성
(광고) 강남역 18년 최대규모의 [임플란트] 치과 6월 이벤트 진행중
가입첫입금15％ 매일첫입금10％ 가입~쿠폰 2만 지급
&quot;2019 자율주행 기술 및 전동화 기술세미나&quot;(07.23)
(광고) 유망사업아이템 &#39;8가지&#39;을 소개드립니다.


In [130]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="71c5f16fa44b8644bf414fa839b9ea9a")>

In [147]:
driver.find_element_by_css_selector(".user_mycafe_list")]

SyntaxError: invalid syntax (<ipython-input-147-8a0670c23356>, line 1)

In [156]:
htms = session.get("https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=10&articleCount=3&useMyCafeEvent=true")

In [162]:
for _ in htms.json()["message"]["result"]["myCafe"]["cafes"]:
    print(_["cafeName"])

스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
독편사-편입,수능,편입영어,학원,학은제,학점은행,학사편입,기출
중고나라
리퍼비쉬팩토리 A/S
수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
릴 베이퍼 궐련형 전자담배 글로 아이코스 csv 쥴 팟 시드 카페
데이터베이스(DB) 전문가 포럼
아사모 -[아이폰, 아이폰 Xr, Xs, Xs Max, 아이패드, 애플워치4]
NSB 농구는 신장이 아니라 심장으로 하는 것이다 NSB 농심카페
구글플레이 고객지원 커뮤니티


In [152]:
for _ in driver.find_elements_by_xpath("//div[@class='user_mycafe_list'] //div[@class='user_mycafe_info']/a/strong[@class='name']"):
    print(_.text)

스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
독편사-편입,수능,편입영어,학원,학은제,학점은행,학사편입,기출
중고나라
리퍼비쉬팩토리 A/S
수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
릴 베이퍼 궐련형 전자담배 글로 아이코스 csv 쥴 팟 시드 카페
데이터베이스(DB) 전문가 포럼
아사모 -[아이폰, 아이폰 Xr, Xs, Xs Max, 아이패드, 애플워치4]
NSB 농구는 신장이 아니라 심장으로 하는 것이다 NSB 농심카페
구글플레이 고객지원 커뮤니티


In [166]:
#로그아웃
driver.find_element_by_css_selector("button.btn_login").click()   #형제노드 찾기

In [169]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#gnb_name2 + a.gnb_btn_login")

[<a class="gnb_btn_login" href="https://nid.naver.com/nidlogin.logout?returl=https://section.cafe.naver.com/Logout.nhn?alreadyNaverLogout=true" id="gnb_logout_button"><span class="gnb_bg"></span><span class="gnb_bdr"></span><span class="gnb_txt">로그아웃</span></a>]

In [170]:
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login ")

<selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="dd66c994-f739-469b-a924-07cd979b091a")>

# 뉴스를 DB에 넣기

In [254]:
driver_list = []

for _ in driver.find_elements_by_xpath("//div[@class='main_component droppable'] //div[@class='hdline_article_tit']"):
    print(_.text)
    driver_list.append(_)

for _ in driver.find_elements_by_xpath("//div[@class='main_component droppable'] //div[@class='com_list']"):
    print(_.text)
    driver_list.append(_)



타다 등 플랫폼 택시 합법화… 수익 일부, 기여금으로 내야
표창원 경찰 출석 "한국당도 법 위에 군림 안 돼"…패스트트랙 줄소환 현실화
靑, 조선·중앙 보도에 문제 제기…"무엇이 한국 위하는 일인가"
정미경 "세월호 단어 들어가면 막말인가…쓴소리 족쇄 채우나"
황교안, '5.18 망언' 김순례 복귀 반대요구 묵살? "법률가 해석 듣고 결정"

윤곽 드러내는 北외무성 새 대미라인…부상에 '뉴페이스' 리태성
"매국적" 조국에 이어 고민정도... 연달아 조선·중앙 보도 비판 포토 오마이뉴스
조국 페이스북 뒷수습?…靑 조선·중앙일보 콕 집어 "국민들 목소리 반영한 것인지 묻는 것" 디지털타임스
국정원 "금강산관광 재개는 유엔제재상 '벌크캐시' 문제와 연결" 포토 연합뉴스
[영상]'단식농성' 지나친 손학규 "새 혁신위원장 못 구해" 포토 노컷뉴스
잠망경 추정물 수색 종료…軍 "어망 부표로 추정" TV조선
일 불매운동 타깃된 유니클로 “임원 발언 죄송” 사과했지만…
[취재파일] 아침밥이란 무엇인가 포토 SBS
김영주 무협 회장 "日 화이트리스트 제외 시 850개 품목 영향…양국 경제에 손해" 포토 아시아경제
韓 경제 `비관론` 확산…日 경제보복에 뚝뚝 떨어지는 기업 실적 눈높이 포토 디지털타임스
3기 신도시 교통 대책이 스마티 모빌리티? 포토 아시아경제
설 자리 잃은 '타다'…플랫폼 택시 허용됐지만 "사업 접을 판" 포토 뉴스1
"일본 여행 중단한다" 회원수 133만 '네일동' 카페 운영 중단
윤석열 선배 13명, 추가용퇴 얼마나…내주 취임 전후 결정 포토 뉴스1
"서울대 왜소해지니 점거 풀라"는 교수들에 학생들 반발 포토 노컷뉴스
국세청, 강남·홍대 유명 클럽 ‘탈탈’ 턴다…수십억대 추징 예고 포토 국민일보
[비디오 디저트] 예술을 원한다면…"맘껏 돌리세요" MBC
새만금 남북도로 2단계 사업 본격화…토지 보상 착수 포토 연합뉴스
동영상 기사
5호 태풍 '다나스' 북상...태풍 영향 어디까지?
[배한철의 역사의 더께3] 들판에서 신하들과 인사불성으로 취한 세종..성군의

In [264]:
for _ in driver.find_elements_by_xpath("//div[@class='main_component droppable'] //div[@class='com_list']"):
    print(_.text)

고민정 청 대변인, ‘혐한 제목’ 조선일보 일본어판 정면 비판
한ㆍ미 연합훈련 이해한다던 김정은, 이젠 훈련중단 선공 카드 포토 중앙일보
윤석열 이어 차기 서울중앙지검장 '주목'… 8월 초 후속 인사 포토 머니S
윤곽 드러내는 北외무성 새 대미라인…부상에 '뉴페이스' 리태성 포토 연합뉴스
"매국적" 조국에 이어 고민정도... 연달아 조선·중앙 보도 비판 포토 오마이뉴스
조국 페이스북 뒷수습?…靑 조선·중앙일보 콕 집어 "국민들 목소리 반영한 것인지 묻는 것" 디지털타임스
[크립토 Up & Down]비트코인, 2주 만에 1만 달러 반납
'3분 카레' 티셔츠·환타 수영복...식품·패션 협업하는 까닭은 포토 조선비즈
유니클로, "韓 불매운동 오래가지 않을 것" 발언 닷새 만에 사과 포토 뉴스1
한은, 8월 금리인하 전망이 우세한 이유는? 파이낸셜뉴스
클럽 주인·대부업자·유명강사…탈세혐의 163명 조사 연합뉴스TV
중국, 美국채 보유 2년만 최저…日 역전 초읽기 포토 머니투데이
"'무역보복' 일본정부 규탄" 20일 日대사관 앞서 촛불 든다
행인 때려 돈 뺏고, 금은방에서 금팔찌 훔친 4인조 붙잡혀 포토 연합뉴스
공장서 지게차로 보행자 들이받아 숨지게 한 50대 집유 2년 포토 연합뉴스
울산환경운동연합 "공해차단 녹지공간 개발 중단해야" 포토 뉴시스
노무라입깃해파리 발견율 4.3배↑…3년 만에 ‘주의’특보 포토 파이낸셜뉴스
"빚 탕감해줄게" 강도로 위장, 남편 청부살해한 60대 2심도 징역 15년 포토 서울경제
국내 최대 일본여행 카페 '네일동' 운영 중단…불매운동 지지
5호 태풍 '다나스' 북상...태풍 영향 어디까지? YTN
[배한철의 역사의 더께3] 들판에서 신하들과 인사불성으로 취한 세종..성군의 황당한 모습1 포토 매일경제
하루 450㎏ 먹는 코끼리, 기후변화 줄이는 ‘착한 식성’ 포토 한겨레
[단독]전국 27개 수소충전소 안전 점검 받는다..“고장 방지 차원” 포토 ZDNet Korea
한나절 떠나는 ‘한캉스’… 이보다 알뜰할 水 없다! 포토 문화일보
경매 

In [256]:
import re

from string import punctuation

pattern = re.compile(r"[{0}]".format(re.escape(punctuation)))

result = pattern.sub("\"", '"')
result

In [253]:
for _ in driver_list:
    print(_.text)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=75.0.3770.100)


In [241]:
cur.executescript("""
        INSERT INTO title(name) 
        VALUES('aa');
    """)

In [251]:
driver_list

[<selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="4879e701-ff00-46ff-82fb-ceecb909efda")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="b10c0763-6aaa-4f41-b05e-0177f6e03d85")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="0d446c0a-1b2e-40cb-981e-7b5adbab2b0b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="ad8da3dc-7433-43f3-a489-30d316647492")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="fbebde3a-e42b-4f0c-8071-f273007f49df")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="eed78063-e6e1-4c29-a791-ebdd8f0b4c05")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="a0390a2b-4c48-4186-a67d-ed

In [252]:



for _ in driver_list:
    print(_.text)
#     print(str(_.text).replace('\"',""))
#     cur.executescript("\
#         INSERT INTO title(name)\
#         VALUES('"+str(_.text.replace("\"",""))+"');\
#     ")
#     con.commit()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=75.0.3770.100)


In [232]:
driver_list[0]

<selenium.webdriver.remote.webelement.WebElement (session="71c5f16fa44b8644bf414fa839b9ea9a", element="3ed53d84-6e07-42e7-bc80-ab1f846ac24a")>

## 모범답안

In [277]:
import sqlite3
con = sqlite3.connect('news.db')
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        num INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL);
        
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        num INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table1_id INTEGER text NOT NULL,
        content TEXT NOT NULL);
""")
con.commit()

driver.get("https://news.naver.com")

for _ in driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a"):
    if len(_.get_attribute("href")) > 1:
        cur.execute("INSERT INTO table1(title, url) VALUES(?,?)",[_.text, _.get_attribute("href")])
        con.commit()

cur.execute("SELECT * FROM table1")
for _ in cur.fetchall():
    driver.get(_[2])
    
    cur.execute("INSERT INTO table2(table1_id, content) VALUES(?,?)", [_[0], driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()

In [ ]:
driver.get("https://news.naver.com")
N = len([_ for _ in driver.find_element_by_css_selector(".hdline_news .hdline_article_tit >a, .com_list li > a")
         if len(_.get_attribute("href")) > 1])

for i in range(N):
    driver.find_element_by_css_selector("#lnb > ul > li:first-child > a").click()
    news = [_ for _ in driver.find_element_by_css_selector(".hdline_news .hdline_article_tit >a, .com_list li > a") 
            if len(_.get_attribute("href"))>1][i]
    cur.execute("INSERT INTO table1(title, url) VALUES(?,?)",[news.text, news.get_attribute("href")])
    con.commit()
    
    cur.execute("SELECT id FROM table1 WHERE url=?", )

In [ ]:
# driver.implicitly_wait

In [280]:
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import ElementNotVisibleException
wait = WebDriverWait(driver, 5, 0.5, [ElementNotVisibleException])
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
#숨겨진걸 보려면-> until
#보이는걸 숨기려면-> until_not

wait.until(EC.visibility_of_all_elements_located(By.CSS_SELECTOR, "#gnb_name2 + a.gnb_btn_login")
          driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login"))

SyntaxError: invalid syntax (<ipython-input-280-e78d1c8c6c66>, line 11)

In [286]:
from selenium.common.exceptions import TimeoutException
wait = WebDriverWait(driver, 5, 0.5, [TimeoutException])
wait.until(lambda x:x.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").is_displayed())

TimeoutException: Message: 


In [283]:
logout = driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login")

In [284]:
logout.is_displayed()

False

In [289]:
#iframe

driver.get("https://www.naver.com")

In [308]:
for _ in driver.find_elements_by_css_selector(""):
    print(_.text)

In [334]:
iframe = driver.find_elements_by_name("da_iframe_time").get_attribute("href")
#driver.switch_to_frame(iframe)
iframe

AttributeError: 'list' object has no attribute 'get_attribute'

In [331]:
driver.find_element_by_tag_name("img").get_attribute("src")

'https://s.pstatic.net/static/www/mobile/edit/2017/0324/mobile_184353754426.gif'

In [335]:
driver.switch_to_default_content()

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [336]:
driver.window_handles  #동작중인 창

['CDwindow-14753DEEDE49EE86D470A44F3A1D171B']

In [340]:
driver.switch_to_window('CDwindow-14753DEEDE49EE86D470A44F3A1D171B')

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [341]:
driver.get("http://www.kt.com")

In [342]:
driver.find_element_by_css_selector("span.quick > a").click()

In [343]:
driver.window_handles

['CDwindow-14753DEEDE49EE86D470A44F3A1D171B',
 'CDwindow-B6CE5DA89A6B4C152997006FB93A3D73']

In [346]:
driver.switch_to_window(driver.window_handles[-1])

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [351]:
driver.find_element_by_id("userId").send_keys("test")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)


In [348]:
driver.close()

In [349]:
driver.window_handles

['CDwindow-14753DEEDE49EE86D470A44F3A1D171B']

In [350]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="71c5f16fa44b8644bf414fa839b9ea9a")>

In [352]:
from selenium.webdriver.common.keys import Keys

In [354]:
driver.find_element_by_tag_name("body).send_keys(Keys.CONTROL"t")

SyntaxError: invalid syntax (<ipython-input-354-8d39e6737ab6>, line 1)

In [355]:
driver.execute_script("javascript:window.open('about:blank');")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)


In [356]:
driver.switch_to_window(driver.window_handles[1])
driver.get("http://www.naver.com")
driver.switch_to_window(driver.window_handles[0])

IndexError: list index out of range

# 인터파크 침투

In [359]:
driver = webdriver.Chrome()
driver.get("https://ticket.interpark.com")